https://github.com/HSUYUCHAO/2025_MIT-QuantumRing_Remote/blob/main/QuFacto.ipynb

# Quantum Factorization with Less Qubits on Quantum Rings Simulator

This project is based on the paper "[Reducing the Number of Qubits in Quantum Factoring](https://eprint.iacr.org/2024/222.pdf)".

Here we will implement algorithm 1 and algorithm 2 from the paper. The algorithm 1 is approximating multi-product algorithm. The algorithm 2 is the quantum factoring algorithm.

In [1]:
# %pip install QuantumRingsLib==0.9.11 matplotlib==3.10.0

In [2]:
import os
import math
import random
from math import ceil, gcd, log, log2

import matplotlib.pyplot as plt
import numpy as np
from QuantumRingsLib import (
    ClassicalRegister,
    QuantumCircuit,
    QuantumRegister,
    QuantumRingsProvider,
    job_monitor,
)
from sympy import divisors, isprime, mod_inverse, primerange
from dotenv import load_dotenv

In [3]:
load_dotenv() # take environment variables from `.env` file
QUANTUM_RINGS_TOKEN = os.getenv("QUANTUM_RINGS_TOKEN", "")
QUANTUM_RINGS_ACCOUNT_NAME = os.getenv("QUANTUM_RINGS_ACCOUNT_NAME", "")

In [4]:
provider = QuantumRingsProvider(token=QUANTUM_RINGS_TOKEN, name=QUANTUM_RINGS_ACCOUNT_NAME)
backend = provider.get_backend("scarlet_quantum_rings")
print(f"Max Qubits: {provider.active_account()['max_qubits']}")

Max Qubits: 128


## Demo on factorizing 143

The configuration can be used to factorize any number, but here we will factorize $N=143$ as an example.

Note: $s, r, G$, depending on $N$, can be tuned for better performance.

- Algo. 2: Step 1) Select a random number $G < N$ invertible modulo $N$.

In [5]:
#################
# Configuration #
#################

# number to be factored
N = CONST_N = 143

# additional bits to increase success probability of the reduction mod N step
# (the computation of the quotient in the RNS reduction step already succeeds
# with overwhelming probability)
# The algorithm should then succeed with probability around 1 - 2^(-20)
CONST_SUCCESS_PROB = 20

# value for the Ekera-Hastad algorithm, for RSA-2048
# taken from Table 3 in "On post-processing in the quantum algorithm
# for computing short discrete logarithms" (Ekera, 2019)
# compute numbers for multi-product
s = 17

# amount of bits of output
r = 22

# we use the RSA variant of Ekera-Hastad: select random G invertible modulo N
G = 2

In [6]:
####################
# Helper functions #
####################


def label(p, i):
    """
    Returns a string representation of a pair (p, i).
    """
    return "(" + str(p) + ", " + str(i) + ")"


def find_rns_base(u, min_bit_size=18):
    """Computes the RNS primes (we don't use 2)."""
    l = list(primerange(2**min_bit_size, 3 * u))
    final_l = []
    final_prod = 1
    for p in l:  # not 2
        final_l.append(p)
        final_prod += log(p, 2)
        if final_prod > u:
            return final_l
    raise Exception("shouldn't stop here")


def plot_histogram(counts, title=""):
    """
    Plots the histogram of the counts

    Args:

        counts (dict):
            The dictionary containing the counts of states

        titles (str):
            A title for the graph.

    Returns:
        None

    """
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]

    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)

    plt.bar(unique, bin_counts)

    maxFreq = max(counts.values())
    plt.ylim(ymax=np.ceil(maxFreq / 10) * 10 if maxFreq % 10 else maxFreq + 10)
    # Show plot
    plt.title(title)
    plt.show()
    return

### Computing the prerequisites of algo. 2.

- Algo. 2: Step 2) Let $A = G^{(N-1)/2-pow(2,n/2-1)}$.
- Algo. 2: Step 3) Precompute $G^{pow(2,i)}$ for $0 \le i < n/2 + n \% 2s +1$ and $A^{pow(2,j)}$ for $0\le n\% 2 + 1$.

Here we denote the base of power series of $G$ as $H$. While the second part of the base of power series of $A$ we need later is just modulo inverse of $G$.
We left the calculation until we need it.

The rest of code in this block is precomputing the cofactors of Algo. 1.

In [7]:
##########################
# Precomputing cofactors #
##########################

n = ceil(log(N, 2))

# compute H = G^((N-1)/2) - 2^((n/2)-1)
H = pow(G, (N - 1) // 2 - 2 ** ((n // 2) - 1), N)
# modular inverse of H for later use
inv_H = mod_inverse(H, N)

# bit-size of N: n bits
print("N=", N)

# maximal bit size of A_X (the product of n/2+ 2l integers smaller than N)
ell = ceil((n / 2) / s)
print("ell=", ell)
m = n // 2 + 2 * ell
print("m=", m)

# bit-size required for the RNS: ceil(log(N,2))*(m/2 + m^(2/3))
max_bit_size = ceil(log(N, 2)) * ceil(m / 2 + pow(m, (2 / 3)))

modN_success_prob_bits = CONST_SUCCESS_PROB
# additional bits to increase success probability of the reduction mod N step
# (the computation of the quotient in the RNS reduction step already succeeds
# with overwhelming probability)
# The algorithm should then succeed with probability around 1 - 2^(-20)

min_bit_size = 2  # of primes in the RNS
rns_base = find_rns_base(max_bit_size, min_bit_size=min_bit_size)
print("Computed RNS base")

# ======================================
M = 1  # rns product
for p in rns_base:
    M *= p

# maximal bit size of primes in the RNS (= w in paper)
max_rns_bit_size = ceil(log(max(rns_base), 2))
# bit size of the RNS product
M_bit_size = ceil(log(M, 2))
# number of primes in RNS
rns_len = len(rns_base)

print("Max represented numbers bit-size:", max_bit_size)
print("Bit-size of RNS product:", M_bit_size)
print("Max bit size of primes in RNS:", max_rns_bit_size)
print("Number of primes in RNS:", rns_len)
_d = {}
for p in rns_base:
    _bit_size = ceil(log(p, 2))
    if _bit_size not in _d:
        _d[_bit_size] = 0
    _d[_bit_size] += 1
print("Distribution of bit-size:", _d)

# setting the size of q_M
alpha = ceil(log(sum(rns_base), 2))
q_M_bit_length = max_rns_bit_size + alpha

# choosing parameter u
_tmp = 0
for p in rns_base:
    _tmp += p**2
u = ceil(log(_tmp, 2)) + 1

# setting parameter u'
bits_in_sum = 0
for p in rns_base:
    bits_in_sum += ceil(log(p, 2))
u_N = ceil(log(bits_in_sum + q_M_bit_length, 2)) + modN_success_prob_bits

# =============================
# computing cofactors for the different steps

print("Computing cofactors...")

# setting parameters for Barrett reduction mod N
barrett_modN_pow2 = n + max_bit_size  # t_N in paper
barrett_modN_reduction_multiplier = (2**barrett_modN_pow2) // N

M_p_dict = {}
M_p_inv_dict = {}
for p in rns_base:
    M_p_dict[p] = M // p
    M_p_inv_dict[p] = mod_inverse(M_p_dict[p], p)

# computing cofactors for the sum for q_M (C_{p,i} in paper)
cofactors_for_qM = {}
for p in rns_base:
    tmp = M_p_inv_dict[p] * (2**u // p)
    for i in range(ceil(log(p, 2))):
        cofactors_for_qM[label(p, i)] = str(
            (2**i * tmp) % (2 ** (u + q_M_bit_length + 1))
        )
print("Computed cofactors for qM")

# first bits of [M_p w_p] for all p
Mp_wp_first_bits = {}
for p in rns_base:
    Mp_wp_first_bits[str(p)] = str(
        ((M_p_dict[p] % (2**r)) * (M_p_inv_dict[p] % (2**r))) % (2**r)
    )

cofactors_for_QN = {}

# cofactors for the bits of q which are in the last sum
tmp = M * barrett_modN_reduction_multiplier
for i in range(q_M_bit_length):
    cofactors_for_QN[label(0, i)] = str(
        (2 ** (u_N + r) - ((tmp) >> (barrett_modN_pow2 - u_N - i))) % 2 ** (u_N + r)
    )

# this is the step which takes most of the time.
done = 0
for p in rns_base:
    done += 1
    if done % 1000 == 0:
        print("Done: ", done, "/", len(rns_base))
    tmp = M_p_dict[p] * M_p_inv_dict[p] * barrett_modN_reduction_multiplier
    for i in range(ceil(log(p, 2))):
        cofactors_for_QN[label(p, i)] = str(
            ((tmp) >> (barrett_modN_pow2 - u_N - i)) % 2 ** (u_N + r)
        )

N= 143
ell= 1
m= 6
Computed RNS base
Max represented numbers bit-size: 56
Bit-size of RNS product: 57
Max bit size of primes in RNS: 6
Number of primes in RNS: 13
Distribution of bit-size: {3: 2, 4: 2, 5: 5, 6: 4}
Computing cofactors...
Computed cofactors for qM


### Algo. 1

Since Algo. 1 is used to checking the probability of error to see the configuration is suitable for Algo. 2, we will not get into the details of Algo. 1.

- Algo. 2:  Step 4) Setup Algo. 1 so that its probability of error p satisfies: $8\sqrt{4\mu}\sqrt{p} < 0.08$.

In [8]:
#######################################
# Approximate multi-product algorithm #
#    (check 8\sqrt(4\mu p) < 0.08)    #
#######################################

N_first_bits = N % (2**r)
M_first_bits = M % (2**r)
repetitions = 100


# the numbers to multiply
A_numbers = ([pow(G, 2**i, N) for i in range(n // 2 + ell)] +
                [pow(H, -2**i, N) for i in range(ell)])
A_numbers_mod_rns = {}
for p in rns_base:
    A_numbers_mod_rns[p] = [(A_numbers[i] % p) for i in range(m)]

count_fail = 0
print("Starting!")
for ctr in range(repetitions):
    # initial value of control bits
    x_initial = [random.randrange(2) for i in range(m)]

    #=======================
    # for the test: compute the actual value
    A_initial = 1
    for i in range(m):
        if x_initial[i]:
            A_initial *= A_numbers[i]
            A_initial = A_initial % N

    # the value we want
    res_real = A_initial % (2**r)

    q_M = 0  # same notations as in Algorithm 1 in the paper
    Q_N = 0
    res = 0

    for p in rns_base:
        # compute the RNS residue
        rns_residue = 1
        for i in range(m):
            # controlled modular product
            if x_initial[i]:
                rns_residue = rns_residue * A_numbers_mod_rns[p][i]
                rns_residue = (rns_residue % p)

        for i in range(ceil(log(p, 2))):
            # controlled additions
            if ((rns_residue >> i) % 2):
                q_M += int(cofactors_for_qM[label(p, i)])
                Q_N += int(cofactors_for_QN[label(p, i)])

        res = (res + rns_residue * int(Mp_wp_first_bits[str(p)])) % (2**r)

    q_M = q_M >> u
    q_M += 1

    res = (res - (q_M * M_first_bits)) % (2**r)
    # at this point, res is zero most of the time

    # finish the computation of Q_N
    for i in range(q_M_bit_length):
        Q_N += (((q_M) >> i) % 2) * int(cofactors_for_QN[label(0, i)])
    # shift the result to get the wanted bits
    Q_N = Q_N >> u_N

    res = (res - (Q_N * N_first_bits)) % (2**r)
    count_fail += int(res != res_real)

print("error rate:", count_fail / repetitions)

Starting!
error rate: 0.0


Check if error rate is below the threshold. Then do the following parts:

- Algo. 2: Step 5) Define the multi-product circuit for $A_0, ..., A_{m-1} = G^{pow(2, 0)}, ..., G^{pow(2, (n/2+n\%2s))}, A^{-pow(2, 0)}, ..., A^{-pow(2, n\%2s)}$.

In [9]:
A = [pow(G, 2**i, N) for i in range(n)]
_A = [pow(inv_H, 2**i, N) for i in range(ceil(n / 2 / s))]
A.extend(_A)
A = list(set(A))  # remove duplicates
A

[2, 4, 42, 16, 113, 48, 18]

### Quantum subroutines of Algo. 2

Here is the main part of Algo. 2.

While $\vert Y \vert < \mu$ do:
- Algo. 2: Step 6) Select a random non-zero mask $\beta$ of $r$ bits.
- Algo. 2: Step 7) Run the "compressed Ekera-Hastad" sub-routine with the following compression of the multi-product circuit:

$$
(x_0, ..., x_{m-1}) \rightarrow \beta \cdot (\Pi A_i^{x_i} \mod N \mod 2^r)
$$.

- Algo. 2: Step 8) Measure $y$.
- Algo. 2: Step 9) If $y\ne 0$, $Y\leftarrow Y \cup \{y\}$.

### Post-processing of Algo. 2

- Algo. 2: Step 10) Run the Ekera-Hastad post-processing routine on Y.
- Algo. 2: Step 11) Let $D$ be the discrete logarithm of $A$. From $D = \frac{P-1}{2} + \frac{Q-1}{2} + 2^{n/2-1}, deduce $P$ and $Q$.

Here we will get the factors $P, Q$ of $N=P Q$.

In [10]:
#####################
# Quantum algorithm #
#####################


def iqft_cct(qc, b, n):
    """
    The inverse QFT circuit

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        n (int):
                The number of qubits in the registers to use

    Returns:
        None

    """

    for i in range(n):
        for j in range(1, i + 1):
            # for inverse transform, we have to use negative angles
            qc.cu1(-math.pi / 2 ** (i - j + 1), b[j - 1], b[i])
        # the H transform should be done after the rotations
        qc.h(b[i])
    qc.barrier()
    return


def find_generator(p):
    """Finds a multiplicative generator for Z_p^* (the multiplicative group modulo p)."""
    if not isprime(p):
        # If p is not prime (e.g., N=143), find a random coprime base
        for g in range(2, p):
            if gcd(g, p) == 1:
                return g
    else:
        div = divisors(p - 1)[:-1]  # Find divisors of p-1
        for i in range(3, p):
            if all(pow(i, d, p) != 1 for d in div):
                return i  # Return the first valid generator
    raise Exception("Couldn't find generator")


class ControlledModularProduct(QuantumCircuit):
    """Quantum circuit for controlled modular multiplication."""

    def __init__(self, A_numbers: list[int], N: int, controlled=True):
        self.N = N
        self.A_numbers = A_numbers
        log2N = ceil(log2(N))

        # Define quantum registers
        controls = QuantumRegister(len(A_numbers), "controls")  # Control qubits
        output = QuantumRegister(ceil(log2(N)), "output")  # Output qubits
        ancillas = QuantumRegister(1, "anc")  # Ancilla qubits

        # Initialize parent class
        super().__init__(controls, output, ancillas, name="controlled_mod_prod")

        self._controls = controls
        self._output = output
        self._ancillas = ancillas

        self.out_size = output.size()

        if controlled:
            self._build_controlled_circuit()

    def _build_controlled_circuit(self):
        """Constructs the controlled modular multiplication circuit."""
        product = 1
        for j, a_num in enumerate(self.A_numbers):
            # Apply controlled modular multiplication
            binary_rep = format(a_num % self.N, f"0{self.out_size}b")

            for k, bit in enumerate(binary_rep[::-1]):
                if bit == "1":
                    self.mcx([self._controls[j]], self._output[k])

            # Add modular reduction
            if j < len(self.A_numbers) - 1:
                self._apply_modular_reduction()

    def _apply_modular_reduction(self):
        """Apply modular reduction after each multiplication."""
        # Basic modular reduction using comparator and subtraction
        n = self.out_size
        N_bin = format(self.N, f"0{n}b")

        # Compare with N and subtract if necessary
        for i in range(n):
            if N_bin[i] == "1":
                self.cx(self._output[i], self._ancillas[0])


def create_compressed_circuit(A_list, beta, N=143, r=22):
    """Creates the compressed quantum circuit as described in Algorithm 2."""
    m = len(A_list)
    n = ceil(log2(N))

    # Define quantum registers
    control = QuantumRegister(m, "ctrl")  # Control qubits
    output = QuantumRegister(ceil(log2(N)), "out")  # Output register
    ancilla = QuantumRegister(1, "anc")  # Ancilla register
    cr = ClassicalRegister(ceil(log2(N)), "c")  # Classical register for measurement

    qc = QuantumCircuit(control, output, ancilla, cr)

    # Apply Hadamard gates to control qubits for superposition
    for i in range(m):
        qc.h(control[i])

    # Add controlled modular multiplication
    cmp = ControlledModularProduct(A_list, N)
    qc.compose(cmp, inplace=True)

    # Apply the beta mask
    beta_bits = format(beta % (2**n), f"0{n}b")
    for i, bit in enumerate(beta_bits):
        if bit == "1":
            qc.cx(control[i % len(control)], output[i])

    # Apply inverse Quantum Fourier Transform (QFT)
    # qc.append(QFT(m, do_swaps=False).inverse(), control[:])
    iqft_cct(qc, control, m)
    qc.measure(output, cr)

    return qc


def run_quantum_subroutine(A_list, mu=20, r=22, N=143):
    """Runs the quantum subroutine and collects measurement results."""
    Y = set()

    qubits_num = 0
    gate_num = 0
    while len(Y) < mu:
        # Generate random non-zero mask with error probability check
        beta = random.randrange(1, 2**10)

        max_attempts = 100
        attempts = 0

        while len(Y) < mu and attempts < max_attempts:
            beta = random.randrange(1, 2**10)
            if beta / np.sqrt(ceil(log2(N))) > 0.08:
                attempts += 1
                continue
            circuit = create_compressed_circuit(A_list, beta, N, r)

        circuit = create_compressed_circuit(A_list, beta, N, r)

        qubits_num = max(qubits_num, circuit.num_qubits + circuit.num_ancillas)
        gate_num = max(gate_num, circuit.size())

        job = backend.run(circuit, shots=100)
        job_monitor(job)
        result = job.result()
        counts = result.get_counts()

        measured = [int(key, 2) for key in counts.keys()]
        for m in measured:
            if m not in Y:  # Ensure unique values are stored
                Y.add(m)
    print("Qubits number: ", qubits_num)
    print("Gate number: ", gate_num)
    return Y


def extract_factors(Y, N):
    """Extract factors from measurement results Y."""
    factors = []
    for y in Y:
        # Try to find factors using GCD
        possible_factor = gcd(y, N)
        if 1 < possible_factor < N:
            factors.append(possible_factor)

    return list(set(factors))  # Remove duplicates


def factor():
    """Main factoring function for N=143 using the quantum algorithm."""
    global A, N, r

    # Run quantum subroutine to get measurements
    Y = run_quantum_subroutine(A, r=r, N=N)
    print("Measurements Y:", Y)

    # Extract factors from measurements
    factors = extract_factors(Y, N)
    if factors:
        print(f"Found factors of {N}:", factors)
        return factors
    else:
        print("No factors found. Try running the algorithm again.")
        return None

In [11]:
%%time
factor()

Job Running
Job Done.
Ending Job Monitor
Job Running
Job Done.
Ending Job Monitor
Qubits number:  16
Gate number:  92
Measurements Y: {11, 13, 15, 40, 42, 44, 46, 56, 58, 60, 62, 73, 75, 77, 79, 82, 89, 91, 93, 95, 101, 103, 104, 106, 120, 122, 124, 126}
Found factors of 143: [11, 13]
CPU times: total: 1.2 s
Wall time: 2.03 s


[11, 13]

Here we obtain $11, 13$ as the factors of $143$.

Leveraging the Quantum Rings simulator, we can get the results from quantum circuits faster than local simulators.

## References

Some of codes are from the awesome [repository](https://gitlab.inria.fr/capsule/quantum-factoring-less-qubits) which is the official implementation of the paper "[Reducing the Number of Qubits in Quantum Factoring](https://eprint.iacr.org/2024/222.pdf)".